# Введение в обработку текста на естественном языке

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Введение в обработку текста на естественном языке"
* https://www.nltk.org/api/nltk.metrics.distance.html
* https://pymorphy2.readthedocs.io/en/stable/user/guide.html
* https://realpython.com/nltk-nlp-python/
* https://scikit-learn.org/stable/modules/feature_extraction.html

In [1]:
!pip install pymorphy2

You should consider upgrading via the 'C:\Users\User\PycharmProjects\mak3\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## Лабораторная работа 7

1\. Загрузите данные из файла `ru_recipes_sample.csv` в виде `pd.DataFrame` `recipes` Используя регулярные выражения, удалите из описаний (столбец `description`) все символы, кроме русских букв, цифр и пробелов. Приведите все слова в описании к нижнему регистру. Сохраните полученный результат в столбец `description`.

In [8]:
import pandas as pd
import re
ruRecipesSample = pd.read_csv("ru_recipes_sample.csv")
r = re.compile(r"[^a-яA-я 1-9]*")
ruRecipesSample['description'] = ruRecipesSample['description'].apply(lambda a: re.sub(r, "", a))
ruRecipesSample['description']

0       Этот коктейль готовлю из замороженной клубники...
1                                         Быстро и вкусно
2                Сытный овощной салатик пальчики оближете
3       Картофельное пюре и куриные котлеты  вкусная к...
4       Вишневая наливка имеет яркий вишневый вкус кот...
                              ...                        
3462    Для тех кто любит чечевицу Вам сюда Очень вкус...
3463    Баклажановые фантазии продолжаются Предлагаю В...
3464    Мое любимое блюдо лазанья Но кушать только фар...
3465    Прошлым летом варила варенье из одуванчиков по...
3466     И три корочки хлеба  сделал заказ Буратино в ...
Name: description, Length: 3467, dtype: object

### Расстояние редактирования

2\. Получите набор уникальных слов `words`, содержащихся в текстах описаний рецептов (воспользуйтесь `word_tokenize` из `nltk`). Сгенерируйте 5 пар случайно выбранных слов и посчитайте между ними расстояние Левенштейна. Выведите на экран результат в следующем виде:

```
d(word1, word2) = x
```

In [9]:
import nltk
nltk.download("punkt")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [20]:
import random
words = set()
for i in ruRecipesSample["description"]:
    for j in word_tokenize(i):
        words.add(j)
words = list(words)
for i in range(5):
    word1 = random.choice(words)
    word2 = random.choice(words)
    print(f"d({word1}, {word2}) = {nltk.edit_distance(word1, word2)}")

d(кедровых, Шарики) = 7
d(модифицировала, Чудесный) = 13
d(каждодневного, покушать) = 12
d(гарнирыв, свечe) = 8
d(весне, веке) = 2


3\. Напишите функцию, которая принимает на вход 2 текстовые строки `s1` и `s2` и при помощи расстояния Левенштейна определяет, является ли строка `s2` плагиатом `s1`. Функция должна реализовывать следующую логику: для каждого слова `w1` из `s1` проверяет, есть в `s2` хотя бы одно слово `w2`, такое, что расстояние Левенштейна между `w1` и `w2` меньше 2, и считает количество таких слов в `s1` $P$.

$$ P = \#\{w_1 \in s_1\ | \exists w_2 \in s_2 : d(w_1, w_2) < tol\}$$

$$ L = max(|s1|, |s2|) $$

Здесь $|\cdot|$ - количество слов в строке, $\#A$ - число элементов в множестве $A$, $w \in s$ означает, что слово $w$ содержится в тексте $s$.

Если отношение $P / L$ больше 0.8, то функция должна вернуть True; иначе False.

Продемонстрируйте работу вашей функции на примере описаний двух рецептов с ID 135488 и 851934 (ID рецепта - это число, стоящее в конце url рецепта). Выведите на экран описания этих рецептов и результат работы функции.

In [11]:
def is_plagiarism(s1: str, s2: str) -> bool:
    words1 = word_tokenize(s1)
    words2 = word_tokenize(s2)
    p = 0
    l = max(len(words1), len(words2))
    for i in words2:
        for j in words1:
            if nltk.edit_distance(i, j) < 2:
                p +=1
                break
    return True if p/l > 0.8 else False

data3 = ruRecipesSample.where(ruRecipesSample['url'].apply(lambda a: a.split("/")[-2] in ['135488', '851934'])).dropna()
is_plagiarism(data3.iloc[0]['description'], data3.iloc[1]['description'])

True

### Стемминг, лемматизация

4\. На основе набора слов из задания 2 создайте `pd.DataFrame` со столбцами `word`, `stemmed_word` и `normalized_word`. В столбец `stemmed_word` поместите версию слова после проведения процедуры стемминга; в столбец `normalized_word` поместите версию слова после проведения процедуры лемматизации. Столбец `word` укажите в качестве индекса. 

Для стемминга можно воспользоваться `SnowballStemmer` из `nltk`, для лемматизации слов - пакетом `pymorphy2`. Сравните результаты стемминга и лемматизации. Поясните на примере одной из строк получившегося фрейма (в виде текстового комментария), в чем разница между двумя этими подходами.

In [12]:
from nltk.stem import SnowballStemmer
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
snb_stemmer_ru = SnowballStemmer('russian')
wordsDf = pd.DataFrame(words, columns=['words'])
wordsDf['stemmed_word'] = wordsDf["words"].apply(lambda a: snb_stemmer_ru.stem(a))
wordsDf['normalized_word'] = wordsDf["words"].apply(lambda a: morph.normal_forms(a)[0])
wordsDf.index = wordsDf["words"]
wordsDf.drop(columns=["words"], inplace=True)
wordsDf

,stemmed_word,normalized_word
words,,
ужин,ужин,ужин
присутствовала,присутствова,присутствовать
лепту,лепт,лепта
Белебеевский,белебеевск,белебеевский
целом,цел,целое
...,...,...
Италию,итал,италия
палочке,палочк,палочка
треской,треск,треска


5\. Добавьте в таблицу `recipes` столбец `description_no_stopwords`, в котором содержится текст описания рецепта после удаления из него стоп-слов. Посчитайте и выведите на экран долю стоп-слов среди общего количества слов. Сравните топ-10 самых часто употребляемых слов до и после удаления стоп-слов.

In [13]:
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from razdel import tokenize

cws = 0
cw = 0
def pop_stop_words(st):
    wrds = word_tokenize(st)
    retWrds = []
    for i in wrds:
        if i.lower() not in ru_stop_words:
            retWrds.append(i)
    return ' '.join(retWrds)

ru_stop_words = stopwords.words("russian")
ruRecipesSample["description_no_stopwords"] = ruRecipesSample["description"].apply(lambda a: pop_stop_words(a))
count_words = sum(ruRecipesSample["description"].apply(lambda a: len(word_tokenize(a))))
count_no_stopwords = sum(ruRecipesSample["description_no_stopwords"].apply(lambda a: len(word_tokenize(a))))
print(f'Процент стоп слов: {(count_words - count_no_stopwords)/count_words* 100}')

words5 = ""
for i in ruRecipesSample["description"]:
    words5 += i.lower() + " "
words5_2 = ""
for i in ruRecipesSample["description_no_stopwords"]:
    words5_2 += i.lower() + " "
print(FreqDist(word_tokenize(words5)).most_common(10))
print(FreqDist(word_tokenize(words5_2)).most_common(10))

Процент стоп слов: 32.03700830165257
[('и', 5043), ('в', 2566), ('с', 1930), ('на', 1642), ('очень', 1594), ('не', 1517), ('из', 1005), ('я', 972), ('а', 849), ('рецепт', 843)]
[('очень', 1594), ('рецепт', 843), ('это', 728), ('блюдо', 521), ('вкусный', 459), ('просто', 434), ('вкусно', 366), ('приготовить', 342), ('вкус', 318), ('салат', 312)]


    ### Векторное представление текста

6\. Выберите случайным образом 5 рецептов из набора данных, в названии которых есть слово "оладьи" (без учета регистра). Представьте описание каждого рецепта в виде числового вектора при помощи `TfidfVectorizer`. На основе полученных векторов создайте `pd.DataFrame`, в котором названия колонок соответствуют словам из словаря объекта-векторизатора. 

Примечание: обратите внимание на порядок слов при создании колонок.

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
c = 0
data6 = []
for i in range(ruRecipesSample["name"].size):
    if "оладьи" in ruRecipesSample["name"][i].lower():
        data6.append(ruRecipesSample["description"][i])
        c += 1
        if c== 5:
            break

tv = TfidfVectorizer()
corpus_tv = tv.fit_transform(data6)
df6 = pd.DataFrame(corpus_tv.toarray(), columns=tv.get_feature_names_out())
df6

,2мл,близких,блины,блюдо,брокколи,был,бюджетное,везде,вкус,вкусно,...,урожай,фарша,хороший,хорошо,цитрусом,что,шалот,это,этом,этот
0,0.136826,0.000000,0.000000,0.000000,0.000000,0.110391,0.000000,0.136826,0.110391,0.000000,...,0.136826,0.136826,0.136826,0.136826,0.000000,0.110391,0.136826,0.000000,0.136826,0.000000
1,0.000000,0.306413,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.247212,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.192788,0.000000,0.000000,0.155540,0.000000,0.000000,0.000000,0.155540,...,0.000000,0.000000,0.000000,0.000000,0.192788,0.155540,0.000000,0.385576,0.000000,0.192788
3,0.000000,0.000000,0.000000,0.306413,0.000000,0.000000,0.306413,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.259428,0.000000,0.000000,0.000000,0.000000,0.209305,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
